# Creating a custom Xarray index 

This tutorial demonstrates [Xarray's](https://xarray.dev/) (relatively) new Flexible Index feature which allows users to modify traditional Xarray indexes to add custom functionality. 

Indexes are an important element of any Xarray object. They facilitate the label-based indexing that makes Xarray a great tool for n-dimensional array data. Most Xarray objects have `Pandas.Indexes`, which fit a wide range of use cases. However, these indexes also have limitations:  
- All coordinate labels must be explicitly loaded in memory,   
- It can be difficult to fit irregularly-sampled data within the `Pandas.Index` structure,  
- There is no built-in support for dimensions that require additional metadata.  

Xarray's custom (wc: flexible?) index feature allows users to define their own Indexes and add them to Xarray objects. A few examples of situations where this is useful are:  
- Periodic index, for datasets with periodic dimensions (such as longitude).   
- Unit-aware index (see the [Pint] project)  
- An index for coordinates described by a function rather than an array  
- An index that can handle a 2D rotation  
(add links to those that have references/examples out there)  

## Overview
We will focus on the following example:  
- We have a 1-dimensional `Xarray.Dataset` indexed in a given coordinate system. However, we want to frequently query the dataset from a different coordinate reference system.  
- Information describing the transformation between the two coordinate systems is stored as an attribute of the Xarray object. In this example, we use a simple, multiplicative transform.
- We want to define a custom index that will handle the coordinate transformation. This is a simplified analog of a common scenario: a geospatial dataset is in a given coordinate reference system and you would like to query it in another coordinate system. (maybe take out last sentence)
- link to existing documentation

We start by defining a very simple index and then increase the complexity by adding more functionality.

![coord transform](img2.png)

## Learning goals
This notebook shows how to build a custom Xarray index and assign it to an Xarray object using [`xr.set_xindex()`](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.set_xindex.html). After working through this tutorial, users should expect to understand:
- How to define a custom Xarray index 
- How to add a custom index index to an existing Xarray object using `xr.set_xindex()`
- The different components of an Xarray index and their function.
- How to implement methods to Xarray indexes such as `.sel()` and methods that handle alignment.



In [1]:
import xarray as xr 
import numpy as np
from collections.abc import Sequence
from copy import deepcopy


from xarray import Index
from xarray.core.indexes import PandasIndex
from xarray.core.indexing import merge_sel_results
from xarray.core.indexes import Index, PandasIndex, get_indexer_nd
from xarray.core.indexing import merge_sel_results

### Define sample data
First, we define a sample dataset to work with. The functions below define parameters that are used to generate an Xarray dataset with a data variable that exists at arbitrary coordinates along an `x` dimension. It also has a scalar variable, `spatial_ref`, where metadata describing the coordinate transform is stored as an attribute. 

In [2]:
def make_kwargs(factor, range_ls, data_len):
    """
    Create keyword arguments for a function.

        Parameters
        ----------
        factor : int or float
            Multiplicative factor for coordinate transform
        range_ls : list
            Range describing the x-coordinate
        data_len : int
            Length of dataset along x-dim

        Returns
        -------
        dict

        """
    da_kwargs = {
        'factor': factor,
        'range' : range_ls,
        'idx_name':'x',
        'real_name':'lon',
        'data_len': data_len
    }
    return da_kwargs

def create_sample_data(kwargs: dict) -> xr.Dataset:
    """
    Function to create sample data.

    Parameters
    ----------
    kwargs : dict
        A dictionary generated from make_kwargs() containing the following key-value pairs:
        - 'factor' (float): A multiplicative factor.
        - 'range' (tuple): A tuple specifying the range of the x-coordinate.
        - 'idx_name' (str): The name of the coordinate reference system A.
        - 'real_name' (str): The name of the coordinate reference system B.

    Returns
    -------
    xr.Dataset
        An Xarray dataset containing the sample data.

    Notes
    -----
    This function creates an Xarray dataset with random data. The dimensions and coordinates of the dataset are specified by the input arguments.

    Example
    -------
    >>> kwargs = {
    ...     'factor': 2.0,
    ...     'range': (0, 10, 1),
    ...     'idx_name': 'coord_A',
    ...     'real_name': 'coord_B'
    ... }
    >>> dataset = create_sample_data(kwargs)
    """
    attrs = {
        'factor': kwargs['factor'],
        'range': kwargs['range'],
        'idx_name': kwargs['idx_name'],
        'real_name': kwargs['real_name']
    }

    da = xr.DataArray(
        data=np.random.rand(kwargs['data_len']),
        dims=(kwargs['idx_name']),
        coords={
            'x': np.arange(kwargs['range'][0], kwargs['range'][1], kwargs['range'][2])
        }
    )

    ds = xr.Dataset({'var1': da})

    spatial_ref = xr.DataArray()
    spatial_ref.attrs = attrs

    ds['spatial_ref'] = spatial_ref
    ds = ds.set_coords('spatial_ref')

    #ds = ds.expand_dims({'y': 1})

    return ds

In [3]:
#create sample data 
sample_ds1 = create_sample_data(make_kwargs(2,[0,10,1],10))

In [4]:
sample_ds1

<xarray.Dataset> Size: 168B
Dimensions:      (x: 10)
Coordinates:
  * x            (x) int64 80B 0 1 2 3 4 5 6 7 8 9
    spatial_ref  float64 8B nan
Data variables:
    var1         (x) float64 80B 0.6362 0.6464 0.3027 ... 0.00198 0.2211 0.5605

## Defining a custom index

### First, how will it be used?
Before we get into defining the custom index, it's helpful to see how it will be used. We have the object `sample_ds1`, which has a `PandasIndex`. 

Note, `PandasIndex` is a Xarray wrapper for `Pandas.Index` object <- maybe more detail than necessary

In [5]:
type(sample_ds1.indexes['x'])

pandas.core.indexes.base.Index

We want to replace the `PandasIndex` with our `CustomIndex`. To do this, we'll drop the `x` index from that dataset:   
`sample_ds1 = sample_ds1.drop_indexes('x')`

Once we define the new index, we'll attach it to the Xarray objects using the `xr.set_xindex()` method. This takes the coordinates from the Xarray object used to build the index, and the index class. It will look like this:

`s1 = sample_ds1.set_xindex(['x','spatial_ref'], ToyIndex_scalar)`

Now, let's define the custom index class.

## The smallest `CustomIndex` 

This is an index that contains only the required component of an Xarray index, the `from_variables()` method. It can be successfully added to `ds` but it can't do much beyond that, and it doesn't contain any information about the transform between coordinate reference systems that we're interested in. Still, it's helpful to understand because `from_variables()` is how information gets from `ds` to our new index. `from_variables()` receives information about `ds` from `xr.set_xindex()` and uses it to construct an instance of `CustomIndex`. 

In [6]:
class CustomIndex_tiny(xr.Index): #customindex inherits xarray Index
    def __init__(self, x_indexes, variables=None): 
        
        self.indexes = variables
        self._xindexes = x_indexes 

        self.spatial_ref = variables['spatial_ref']
        
    @classmethod          
    def from_variables(cls,variables, **kwargs):
        '''this method creates a CustomIndex obj from a variables object.
        variables is a dict created from ds1, keys are variable names,
        values are associated xr.variables. created like this:
        coord_vars = {name:ds._variables[name] for name in coord_names}
        coord_names is passed to set_xindex
        '''
        # this index class expects to work with datasets with certain properties
        # it must have exactly 2 variables: x and spatial_ref 
        assert len(variables) == 2
        assert 'x' in variables
        assert 'spatial_ref' in variables 
        
        #separate dimensional, scalar variables into own dicts
        dim_variables = {}
        scalar_vars = {}
        for k,i in variables.items():
            if variables[k].ndim ==1:
                dim_variables[k] = variables[k]
            if variables[k].ndim ==0:
                scalar_vars[k] = variables[k]
        
        options = {'dim':'x',
                   'name':'x'}
        
        #make dict of PandasIndexes for dim. variable
        x_indexes = {
            k: PandasIndex.from_variables({k: v}, options = options) 
            for k,v in dim_variables.items()
        }
        #add scalar var to dict
        x_indexes['spatial_ref'] = variables['spatial_ref']
        
        return cls(x_indexes, variables) #return an instance of CustomIndex class


In [7]:
sample_ds1 = sample_ds1.drop_indexes('x')

In [8]:
ds1 = sample_ds1.set_xindex(['x','spatial_ref'], CustomIndex_tiny)

In [9]:
ds1

<xarray.Dataset> Size: 168B
Dimensions:      (x: 10)
Coordinates:
  * x            (x) int64 80B 0 1 2 3 4 5 6 7 8 9
  * spatial_ref  float64 8B nan
Data variables:
    var1         (x) float64 80B 0.6362 0.6464 0.3027 ... 0.00198 0.2211 0.5605
Indexes:
  ┌ x            CustomIndex_tiny
  └ spatial_ref

As mentioned above, `ds1` now has the CustomIndex, but it can't do much.

In [10]:
%xmode Minimal

ds1.sel(x=4)

Exception reporting mode: Minimal


NotImplementedError: <__main__.CustomIndex_tiny object at 0x7f7c7c220ad0> doesn't support label-based selection

### More detail on `from_variables()`
> - During `xr.set_xindex()`, a dict object called `variables` is created. For every coordinate in `ds`, `variables` has a key-value pair like follows: `name: ds._variables[name]`. 
> - `variables` is passed to `from_variables()` and used to create another dict. The values in this dictionary hold a `PandasIndex` for each dimensional coordinate, and an `xr.Variable` for each scalar coordinate. 
> - It's important to note that `from_variables()` is a **class method** (Add link). This means that it acts as a constructor, returning an instance of the `CustomIndex` class. 




## Adding a coordinate transform and `.sel()` to `CustomIndex`
This section adds three new methods:
1. `create_variables()`: Returns a coordinate variable created from the new index.
2. `transform()`: Handles the coordinate transform between CRS A and CRS B. <- NOTE: remove this from class and pass to set_xindex?
3. `sel()`: Select points from `ds1` using `transform()`. This allows user to pass labels in coordinate reference system B, and `.sel()` will return appropriate elements from ds1.

In [11]:
#create new sample data
sample_ds1 = create_sample_data(make_kwargs(2,[0,10,1],10))

#create a copy used for testing later
orig_ds1 = sample_ds1.copy()

In [12]:
sample_ds1

<xarray.Dataset> Size: 168B
Dimensions:      (x: 10)
Coordinates:
  * x            (x) int64 80B 0 1 2 3 4 5 6 7 8 9
    spatial_ref  float64 8B nan
Data variables:
    var1         (x) float64 80B 0.9233 0.5917 0.4112 ... 0.291 0.2584 0.1781

In [13]:
class CustomIndex_sel(xr.Index): #customindex inherits xarray Index
    def __init__(self, x_indexes, variables=None): 
        
        self.indexes = variables
        self._xindexes = x_indexes 

        self.spatial_ref = variables['spatial_ref']
        
    @classmethod          
    def from_variables(cls,variables, **kwargs):
        '''this method creates a CustomIndex obj from a variables object.
        variables is a dict created from ds1, keys are variable names,
        values are associated xr.variables. created like this:
        coord_vars = {name:ds._variables[name] for name in coord_names}
        coord_names is passed to set_xindex
        '''
        # this index class expects to work with datasets with certain properties
        # it must have exactly 2 variables: x and spatial_ref 
        assert len(variables) == 2
        assert 'x' in variables
        assert 'spatial_ref' in variables 
        
        dim_variables = {}
        scalar_vars = {}
        for k,i in variables.items():
            if variables[k].ndim ==1:
                dim_variables[k] = variables[k]
            if variables[k].ndim ==0:
                scalar_vars[k] = variables[k]
        
        options = {'dim':'x',
                   'name':'x'}
        
        x_indexes = {
            k: PandasIndex.from_variables({k: v}, options = options) 
            for k,v in dim_variables.items()
        }
        
        x_indexes['spatial_ref'] = variables['spatial_ref']
        
        return cls(x_indexes, variables) #return an instance of CustomIndex class
    
    def create_variables(self, variables=None):
        '''
        Creates coord variable from index.

        Parameters:
        -----------
        variables : dict, optional
            A dictionary of variables.

        Returns:
        --------
        dict
            A dictionary containing the created variables.

        Notes:
        ------
        This method iterates over the `_xindexes` values and creates coord variables from the indexes.
        It skips the spatial reference variable and updates the `idx_variables` dictionary with the created variables.
        Finally, it adds the `spatial_ref` variable from the `variables` dictionary to the `idx_variables` dictionary.

        Example:
        --------
        >>> variables = {'spatial_ref': 123}
        >>> result = create_variables(variables)
        >>> print(result)
        {'var1': ..., 'var2': ..., 'spatial_ref': 123}
        '''
        idx_variables = {}

        for index in self._xindexes.values():
            if type(index) == xr.core.variable.Variable:
                pass
            else:
                x = index.create_variables(variables)
                idx_variables.update(x)

        idx_variables['spatial_ref'] = variables['spatial_ref']
        return idx_variables

        idx_variables = {}
        

        for index in self._xindexes.values():
            #want to skip spatial ref
            if type(index) == xr.core.variable.Variable:
                pass
            else:

                x = index.create_variables(variables)
                idx_variables.update(x)
                
        idx_variables['spatial_ref'] = variables['spatial_ref']          
        return idx_variables

    def transform(self, value):
        """
        Transform the given value based on the spatial reference attributes. Currently, this only handles a very simple transform.
        NOTE: this could be removed from the index class and passed to set_xindex()? 

        Parameters:
        -----------
        value : int, float, slice, or list
            The value to be transformed.

        Returns:
        --------
        transformed_labels : dict
            A dictionary containing the transformed labels.

        Notes:
        ------
        - If `value` is a slice, it will be transformed based on the factor and index name attributes.
        - If `value` is a single value or a list of values, each value will be transformed based on the factor attribute.

        Examples:
        ---------
        >>> spatial_ref = SpatialReference(factor=2, idx_name='index')
        >>> transformed_labels = spatial_ref.transform(10)
        >>> print(transformed_labels)
        {'index': 5}

        >>> transformed_labels = spatial_ref.transform([10, 20, 30])
        >>> print(transformed_labels)
        {'index': [5, 10, 15]}

        >>> transformed_labels = spatial_ref.transform(slice(10, 20, 2))
        >>> print(transformed_labels)
        {'index': slice(5, 10, 2)}
        """
        #extract attrs
        fac = self.spatial_ref.attrs['factor']
        key = self.spatial_ref.attrs['idx_name']

        #handle slice
        if isinstance(value, slice):
            
            start, stop, step = value.start, value.stop, value.step
            new_start, new_stop, new_step = start / fac, stop/fac, step
            new_val = slice(new_start, new_stop, new_step)
            transformed_labels = {key: new_val}
            return transformed_labels
        
        #single or list of values
        else:
        
            vals_to_transform = [] 

            if not isinstance(value, Sequence):
                value = [value]

            for k in range(len(value)):

                val = value[k]
                vals_to_transform.append(val)

            #logic for parsing attrs
            transformed_x = [int(v / fac) for v in vals_to_transform]

            transformed_labels = {key:transformed_x}
            return transformed_labels

    def sel(self, labels):
        """
        Selects data from the index based on the provided labels.

        Parameters:
        -----------
        labels : dict
            A dictionary containing the labels for each dimension.

        Returns:
        --------
        matches : PandasIndex
            A PandasIndex object containing the selected data.

        Raises:
        -------
        AssertionError:
            If the type of `labels` is not a dictionary.

        Notes:
        ------
        - The `labels` dictionary should have keys corresponding to the dimensions of the index.
        - The values of the `labels` dictionary should be the labels to select from each dimension.
        - The method uses the `transform` method to convert the labels to coordinate CRS.
        - The selection is performed on the index created in the `.sel()` method.

        Example:
        --------
        >>> labels = {'x': 10}
        >>> matches = obj.sel(labels)
        >>> print(matches)
        PandasIndex([10], dtype='int64', name='x')
        """
        
        assert type(labels) == dict

        #user passes to sel
        label = next(iter(labels.values()))

        #materialize coord array to idx off of
        params = self.spatial_ref.attrs['range']
        full_arr = np.arange(params[0], params[1], params[2])
        toy_index = PandasIndex(full_arr, dim='x')

        #transform user labesl to coord crs
        idx = self.transform(label)

        #sel on index created in .sel()
        matches = toy_index.sel(idx)

        return matches 

Drop index:

In [15]:
sample_ds1 = sample_ds1.drop_indexes('x')

In [16]:
ds1 = sample_ds1.set_xindex(['x','spatial_ref'], CustomIndex_sel)

In [17]:
ds1

<xarray.Dataset> Size: 168B
Dimensions:      (x: 10)
Coordinates:
  * x            (x) int64 80B 0 1 2 3 4 5 6 7 8 9
  * spatial_ref  float64 8B nan
Data variables:
    var1         (x) float64 80B 0.7656 0.8868 0.01945 ... 0.3991 0.5366
Indexes:
  ┌ x            CustomIndex_sel
  └ spatial_ref

Let's see if this works! Remember our coordinate transform (add desc. or illustration)

In [18]:
ds1.sel(x=14)

<xarray.Dataset> Size: 24B
Dimensions:      (x: 1)
Coordinates:
    x            (x) int64 8B 7
  * spatial_ref  float64 8B nan
Data variables:
    var1         (x) float64 8B 0.002764
Indexes:
    spatial_ref  CustomIndex_sel

In [19]:
assert ds1.sel(x=14) == orig_ds1.sel(x=7)

`.sel()` can also handle passing lists and slices

In [20]:
ds1.sel(x=[8,10,14])

<xarray.Dataset> Size: 56B
Dimensions:      (x: 3)
Coordinates:
    x            (x) int64 24B 4 5 7
  * spatial_ref  float64 8B nan
Data variables:
    var1         (x) float64 24B 0.1692 0.1182 0.002764
Indexes:
    spatial_ref  CustomIndex_sel

In [21]:
# dim order switches? so need to specify data to assert
assert np.array_equal(ds1.sel(x=[8,10,14])['var1'].data, orig_ds1.sel(x=[4,5,7])['var1'].data)

In [22]:
ds1.sel(x=slice(4,18))

<xarray.Dataset> Size: 136B
Dimensions:      (x: 8)
Coordinates:
    x            (x) int64 64B 2 3 4 5 6 7 8 9
  * spatial_ref  float64 8B nan
Data variables:
    var1         (x) float64 64B 0.01945 0.9708 0.1692 ... 0.3991 0.5366
Indexes:
    spatial_ref  CustomIndex_sel

In [23]:
assert np.array_equal(ds1.sel(x=slice(4,18))['var1'].data, orig_ds1.sel(x=slice(2,9))['var1'].data)

## Adding align

NOTE: add illustration? 

Alignment is an important capability of Xarray indexes. It relies on three methods: `equals()`, `join()` and `reindex_like()`. 
- `equals()`: Checks if the index is equal to the other index passed in the signatures are equal.
- `join()`: Joins the two indexes.
- `reindex_like()`: Reindexes the current index to match the result of the join.
Let's add them to the index :

In [18]:
class CustomIndex(xr.Index): #customindex inherits xarray Index
    def __init__(self, x_indexes, variables=None): 
        
        self.indexes = variables
        self._xindexes = x_indexes 
        if variables is not None:

            self.spatial_ref = variables['spatial_ref']
        else:
            self.spatial_ref = None
    @classmethod          
    def from_variables(cls,variables, **kwargs):
        '''this method creates a CustomIndex obj from a variables object.
        variables is a dict created from ds1, keys are variable names,
        values are associated xr.variables. created like this:
        coord_vars = {name:ds._variables[name] for name in coord_names}
        coord_names is passed to set_xindex
        '''
        # this index class expects to work with datasets with certain properties
        # must have exactly 2 variables: x and spatial_ref 
        assert len(variables) == 2
        assert 'x' in variables
        assert 'spatial_ref' in variables 
        
        dim_variables = {}
        scalar_vars = {}
        for k,i in variables.items():
            if variables[k].ndim ==1:
                dim_variables[k] = variables[k]
            if variables[k].ndim ==0:
                scalar_vars[k] = variables[k]
        
        options = {'dim':'x',
                   'name':'x'}
        
        x_indexes = {
            k: PandasIndex.from_variables({k: v}, options = options) 
            for k,v in dim_variables.items()
        }
        
        x_indexes['spatial_ref'] = variables['spatial_ref']
        
        return cls(x_indexes, variables)
    
    def create_variables(self, variables=None):
        '''creates coord variable from index'''
        if not variables:
            variables = self.joined_var

        idx_variables = {}
        

        for index in self._xindexes.values():
            #want to skip spatial ref
            if type(index) == xr.core.variable.Variable:
                pass
            else:

                x = index.create_variables(variables)
                idx_variables.update(x)
                
        idx_variables['spatial_ref'] = variables['spatial_ref']          
        return idx_variables

    def transform(self, value):
        
        #extract attrs
        fac = self.spatial_ref.attrs['factor']
        key = self.spatial_ref.attrs['idx_name']

        #handle slice
        if isinstance(value, slice):
            
            start, stop, step = value.start, value.stop, value.step
            new_start, new_stop, new_step = start / fac, stop/fac, step
            new_val = slice(new_start, new_stop, new_step)
            transformed_labels = {key: new_val}
            return transformed_labels
        
        #single or list of values
        else:
        
            vals_to_transform = [] 

            if not isinstance(value, Sequence):
                value = [value]

            for k in range(len(value)):

                val = value[k]
                vals_to_transform.append(val)

            #logic for parsing attrs, todo: switch to actual transform
            transformed_x = [int(v / fac) for v in vals_to_transform]

            transformed_labels = {key:transformed_x}
            return transformed_labels

    def sel(self, labels):
        
        assert type(labels) == dict

        #user passes to sel
        label = next(iter(labels.values()))

        #materialize coord array to idx off of
        params = self.spatial_ref.attrs['range']
        full_arr = np.arange(params[0], params[1], params[2])
        toy_index = PandasIndex(full_arr, dim='x')

        #transform user labesl to coord crs
        idx = self.transform(label)

        #sel on index created in .sel()
        matches = toy_index.sel(idx)

        return matches 
        

    def equals(self, other):
        """
        Check if the current instance is equal to another instance.
        Parameters
        ----------
        other : object
            The other instance to compare with.

        Returns
        -------
        bool
            True if the current instance is equal to the other instance, False otherwise.
       """
        
        result = self._xindexes['x'].equals(other._xindexes['x']) and self._xindexes['spatial_ref'].equals(other._xindexes['spatial_ref'])
        
        return result

    def join(self, other, how='inner'):
        """
        Join the current index with another index.

        Parameters:
        -----------
        other : PandasIndex
            The index to join with.
        how : str, optional
            The type of join to perform. Default is 'inner'.

        Returns:
        --------
        new_obj : PandasIndex
            A new PandasIndex object representing the joined index.

        Notes:
        ------
        This method joins the current index with another index based on a common dimension.

        The current index and the other index are first converted into PandasIndex objects.

        The spatial reference information of the joined index is updated based on the start, stop, and step values of the joined index.

        The joined index is then converted back into a PandasIndex object and returned as a new PandasIndex object.
        """
        #make self index obj
        params_self = self.spatial_ref.attrs['range']
        full_arr_self = np.arange(params_self[0], params_self[1], params_self[2])
        toy_index_self = PandasIndex(full_arr_self, dim='x')
        

        #make other index obj
        other_start = other._xindexes['x'].index.array[0]
        other_stop = other._xindexes['x'].index.array[-1]
        other_step = np.abs(int((other_start-other_stop) / (len(other._xindexes['x'].index.array)-1)))
        
        
        params_other = other.spatial_ref.attrs['range']
        full_arr_other = np.arange(other_start, other_stop, other_step) #prev elements of params_other
        toy_index_other = PandasIndex(full_arr_other, dim='x')
        
        self._indexes = {'x': toy_index_self}
        other._indexes = {'x':toy_index_other}
        
        
        new_indexes = {'x':toy_index_self.join(toy_index_other, how=how)}
        
        #need to return an index obj, but don't want to have to pass variables
        # so need to add all of the things that index needs to new_indexes before passign it to return?
        
        #this will need to be generalized / tested more
        new_indexes['spatial_ref'] =  deepcopy(self.spatial_ref) 
        start = int(new_indexes['x'].index.array[0])
        stop = int(new_indexes['x'].index.array[-1])
        step = int((stop-start) / (len(new_indexes['x'].index.array) -1))
        
        new_indexes['spatial_ref'].attrs['range'] = [start, stop, step]
        
        idx_var = xr.IndexVariable(dims=new_indexes['x'].index.name,
                                   data = new_indexes['x'].index.array)
        attr_var = new_indexes['spatial_ref']
                              
        idx_dict = {'x':idx_var, 
                   'spatial_ref':attr_var}
        
        new_obj = type(self)(new_indexes)
        new_obj.joined_var = idx_dict
        return new_obj
        

    def reindex_like(self, other, method=None, tolerance=None):
        """
        Reindexes the current object to match the index of another object.

        Parameters:
        -----------
        other : object
            The object whose index will be used for reindexing.
        method : str, optional
            The method to use for reindexing. Default is None.
        tolerance : float, optional
            The tolerance value to use for reindexing. Default is None.

        Returns:
        --------
        dict
            A dictionary containing the reindexed values.

        Raises:
        -------
        None

        Notes:
        ------
        This method reindexes the current object to match the index of the `other` object.
        It uses the `method` and `tolerance` parameters to determine the reindexing behavior.
        The reindexed values are returned as a dictionary.
        """
        
        params_self = self.spatial_ref.attrs['range']
        full_arr_self = np.arange(params_self[0], params_self[1], params_self[2])
        toy_index_self = PandasIndex(full_arr_self, dim='x')
       
        toy_index_other = other._xindexes['x']
    
        d = {'x': toy_index_self.index.get_indexer(other._xindexes['x'].index, method, tolerance)}
               
        return d
        

In [19]:
#create new sample data
sample_ds1 = create_sample_data(make_kwargs(2,[0,10,1],10))
sample_ds2 = create_sample_data(make_kwargs(5,[5,15,1],10))


#create a copy used for testing later
orig_ds1 = sample_ds1.copy()
orig_ds2 = sample_ds2.copy()

*** reindex_like needs to return an object like variables to pass to create vars (?)

In [20]:
sample_ds1 = sample_ds1.drop_indexes('x')
sample_ds2 = sample_ds2.drop_indexes('x')

In [21]:
ds1 = sample_ds1.set_xindex(['x','spatial_ref'], CustomIndex)
ds2 = sample_ds2.set_xindex(['x','spatial_ref'], CustomIndex)


## Align

In [22]:
#create sample data -- we define 2 for alignment
sample_ds1 = create_sample_data(make_kwargs(2,[0,10,1],10))
sample_ds2 = create_sample_data(make_kwargs(5,[8,18,1], 10))

#create copies used for testing later
orig_ds1 = sample_ds1.copy()
orig_ds2 = sample_ds2.copy()

In [23]:
sample_ds1 = sample_ds1.drop_indexes('x')
sample_ds2 = sample_ds2.drop_indexes('x')

In [24]:
ds1 = sample_ds1.set_xindex(['x','spatial_ref'], CustomIndex)
ds2 = sample_ds2.set_xindex(['x','spatial_ref'], CustomIndex)


In [25]:
inner_align, _ = xr.align(ds1, ds2, join='inner')

In [26]:
outer_align, _ = xr.align(ds1, ds2, join='outer')

In [27]:
outer_align

<xarray.Dataset> Size: 280B
Dimensions:      (x: 17)
Coordinates:
  * x            (x) int64 136B 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16
  * spatial_ref  float64 8B nan
Data variables:
    var1         (x) float64 136B 0.06861 0.6461 0.5169 0.1095 ... nan nan nan
Indexes:
    x            CustomIndex
    spatial_ref  CustomIndex

In [28]:
inner_align

<xarray.Dataset> Size: 40B
Dimensions:      (x: 2)
Coordinates:
  * x            (x) int64 16B 8 9
  * spatial_ref  float64 8B nan
Data variables:
    var1         (x) float64 16B 0.1917 0.01797
Indexes:
    x            CustomIndex
    spatial_ref  CustomIndex

In [29]:
# reindex_like not implemented for PandasIndx
# but that defaults to inner, and these are successsfuly producing left and right so shouldn't be it
#left_align,_ = xr.align(ds1, ds2, join='left')
#right_align,_ = xr.align(ds1, ds2, join='right')

#don't remember what above was about , is reindex like not implemented for left, right joins something like that ?

## Wrap up / summary
To do